<a href="https://colab.research.google.com/github/vipashaaV321/User-Intent-Modeling/blob/main/MovieLens_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Tensorflow library. Used to implement machine learning models
# import tensorflow as tf

#Numpy contains helpful functions for efficient mathematical calculations
import numpy as np
#Dataframe manipulation library
import pandas as pd
#Graph plotting library
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from IPython.display import SVG
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot

from sklearn.metrics import mean_squared_error as MSE,mean_absolute_error
from tabulate import tabulate
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/Recommendation System/data.csv', names = titles)

In [ ]:
#Loading in the movies dataset
movies_df = pd.read_csv('/content/drive/MyDrive/movie-lens/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
movies_df.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
#Loading in the movies dataset
ratings_df = pd.read_csv('/content/drive/MyDrive/movie-lens/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings_df.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
movies_df.columns = ['MovieID', 'Title', 'Genres']
movies_df.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings_df.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## Simple Deep Neural Network Model

Simple Deep neural network model combining a collaborative filtering recommendation algorithm with deep learning technology, therein consisting of two parts. First, the model uses a feature representation method based on a quadric polynomial regression model,which obtains the latent features more accurately by improving upon the traditional matrix factorization algorithm. Then, these latent features are regarded as the input data of the deep neural network model, which is the second part of the proposed model and is used to predict the rating scores.

In [ ]:
len(ratings_df.UserID.unique()), len(ratings_df.MovieID.unique())

(6040, 3706)

In [ ]:
# We assign a unique number between (0, #users)
# to each user and do the same for movies and create dataframe which have userid and movieid along with newly generated userid and movieid.

ratings_df['UserID_C'] = ratings_df.UserID.astype('category').cat.codes.values
ratings_df['MovieID_C'] = ratings_df.MovieID.astype('category').cat.codes.values
user_movie_cate_df = ratings_df[['UserID','UserID_C','MovieID','MovieID_C']]
ratings_df.drop(['UserID','MovieID'],axis=1,inplace=True)

In [ ]:
ratings_df.head()

,Rating,Timestamp,UserID_C,MovieID_C
0,5,978300760,0,1104
1,3,978302109,0,639
2,3,978301968,0,853
3,4,978300275,0,3177
4,5,978824291,0,2162


In [ ]:
# Train test split
# We'll now split our dataset of ratings into train and test. Given the train set, we'd like to accurately estimate the ratings in the test set.

train, test = train_test_split(ratings_df, test_size=0.2,random_state=7856)

# True rating for test dataframe
y_true = test.Rating

In [ ]:
n_users, n_movies = len(ratings_df.UserID_C.unique()), len(ratings_df.MovieID_C.unique())
train.head()

,Rating,Timestamp,UserID_C,MovieID_C
251405,4,974772836,1518,858
784478,2,965532329,4681,1279
464897,1,972479172,2865,2275
1767,4,978160616,16,1741
266208,4,974740563,1612,1867


# Neural networks for recommendation
We'll now create a simple neural network for recommendation, or for estimating rating! This model is very similar to the matrix factorisation models, but differs in the following ways:

Instead of taking a dot product of the user and the item embedding, we concatenate them and use them as features for our neural network. Thus, we are not constrained to the dot product way of combining the embeddings, and can learn complex non-linear relationships.

Due to #1, we can now have a different dimension of user and movie embeddings. This can be useful if one dimension is larger than the other.

In [ ]:
n_latent_factors_user = 6
n_latent_factors_movie = 10

movie_input = keras.layers.Input(shape=[1],name='Item')
movie_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors_movie, name='Movie-Embedding')(movie_input)
movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)
movie_vec = keras.layers.Dropout(0.2)(movie_vec)


user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors_user,name='User-Embedding')(user_input))
user_vec = keras.layers.Dropout(0.2)(user_vec)


concat = keras.layers.concatenate([movie_vec, user_vec])
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense = keras.layers.Dense(200,name='FullyConnected')(concat)
dropout_1 = keras.layers.Dropout(0.2,name='Dropout')(dense)
dense_2 = keras.layers.Dense(100,name='FullyConnected-1')(concat)
dropout_2 = keras.layers.Dropout(0.2,name='Dropout')(dense_2)
dense_3 = keras.layers.Dense(50,name='FullyConnected-2')(dense_2)
dropout_3 = keras.layers.Dropout(0.2,name='Dropout')(dense_3)
dense_4 = keras.layers.Dense(20,name='FullyConnected-3', activation='relu')(dense_3)


result = keras.layers.Dense(1, activation='relu',name='Activation')(dense_4)
adam = Adam(lr=0.005)
model = keras.Model([user_input, movie_input], result)
model.compile(optimizer=adam,loss= 'mean_absolute_error')

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Item (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 User (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 Movie-Embedding (Embedding)    (None, 1, 10)        37070       ['Item[0][0]']                   
                                                                                                  
 User-Embedding (Embedding)     (None, 1, 6)         36246       ['User[0][0]']                   
                                                                                            

In [ ]:
history = model.fit([train.UserID_C, train.MovieID_C], train.Rating, epochs=30, verbose=0)

In [ ]:
y_hat_2 = np.round(model.predict([test.UserID_C, test.MovieID_C]),0)
print(mean_absolute_error(y_true, y_hat_2))

print(mean_absolute_error(y_true, model.predict([test.UserID_C, test.MovieID_C])))

/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


0.6729486807770368
0.6950170892379126


In [ ]:
test_user_215= pd.DataFrame({'UserID_C' : [214] * 3706,'MovieID_C':list(ratings_df['MovieID_C'].unique())})
#test_user_215.head()
test_user_215['Predicted_Ratings'] = np.round(model.predict([test_user_215.UserID_C, test_user_215.MovieID_C]),0)

In [ ]:
test_user_215.head()

,UserID_C,MovieID_C,Predicted_Ratings
0,214,1104,5.0
1,214,639,5.0
2,214,853,5.0
3,214,3177,5.0
4,214,2162,5.0


In [ ]:
result = pd.merge(test_user_215,test,how='left',on=['UserID_C','MovieID_C'])
result = result[pd.isnull(result['Rating'])]
result.sort_values(by='Predicted_Ratings',ascending=False,inplace=True)
result.head(10)

,UserID_C,MovieID_C,Predicted_Ratings,Rating,Timestamp
0,214,1104,5.0,NaN,NaN
2090,214,899,5.0,NaN,NaN
1992,214,468,5.0,NaN,NaN
1995,214,1213,5.0,NaN,NaN
1997,214,547,5.0,NaN,NaN
1998,214,1931,5.0,NaN,NaN
1999,214,1933,5.0,NaN,NaN
2000,214,1085,5.0,NaN,NaN
2002,214,2997,5.0,NaN,NaN
2003,214,2583,5.0,NaN,NaN


In [ ]:
result.rename(columns={'MovieID_C':'MovieID'},inplace=True)
final_df = pd.merge(result,movies_df,how='inner',on=['MovieID'])
final_df.sort_values(by=['Predicted_Ratings'],ascending=False).head(20)

,UserID_C,MovieID,Predicted_Ratings,Rating,Timestamp,Title,Genres
0,214,1104,5.0,NaN,NaN,"Streetcar Named Desire, A (1951)",Drama
1239,214,2260,5.0,NaN,NaN,Wisdom (1986),Action|Crime
1251,214,2958,5.0,NaN,NaN,Naturally Native (1998),Drama
1250,214,2832,5.0,NaN,NaN,"Lost Son, The (1999)",Drama
1249,214,910,5.0,NaN,NaN,Some Like It Hot (1959),Comedy|Crime
1248,214,537,5.0,NaN,NaN,Sirens (1994),Comedy|Drama
1247,214,188,5.0,NaN,NaN,"Prophecy, The (1995)",Horror
1246,214,1125,5.0,NaN,NaN,"Return of the Pink Panther, The (1974)",Comedy
1245,214,1003,5.0,NaN,NaN,Extreme Measures (1996),Drama|Thriller
1244,214,1849,5.0,NaN,NaN,Prince Valiant (1997),Adventure


# Deep Autoencoder


An Autoencoder is a profound learning neural system design that accomplishes best in class execution in the territory of collaborative filtering and furthermore used to get familiar with a representation (encoding) for a lot of input data, usually to a achieve dimensionality reduction. Architecturally, the type of an Autoencoder is a feedforward neural system having an input layer, one hidden layer and a output layer.The output layer has the same number of neurons as the input layer for the purpose of reconstructing it’s own inputs. It is useful that an Autoencoder has a smaller hidden layer than the input layer. It is useful that an Autoencoder has a smaller hidden layer than the input layer. This effect forces the model to create a compressed representation of the data in the hidden layer by learning correlations in the data.The core purpose of the algorithm is to reconstruct the original input data.During the training time the encoder takes a input data sample x and maps it to the so called hidden or latent representation z. Then the decoder maps z to the output vector x’ which is (in the best case scenario) the exact representation of the input data x. Please notice that usually an exact recreation of the input x is not possible.The additional hidden layers enable the Autoencoder to learn mathematically more complex underlying patterns in the data.Deeper layers of the Deep Autoencoder tend to learn even higher-order features.

In [ ]:
#Loading in the movies dataset
ratings_df = pd.read_csv('/content/drive/MyDrive/movie-lens/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings_df.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
movies_df.columns = ['MovieID', 'Title', 'Genres']
movies_df.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings_df.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
# Lets pivot the data to get it at a user level
ratings_transform = pd.pivot_table(ratings_df[['UserID','MovieID','Rating']],values=['Rating'], index=['UserID'], columns=['MovieID'] ).fillna(0)

# creating train and test sets
X_train_enc, X_test_enc = train_test_split(ratings_transform, train_size=0.8,random_state=7856)

In [ ]:
X_train_enc.head()

Rating                                               ...            \
MovieID   1    2    3    4    5    6    7    8    9    10    ... 3943 3944   
UserID                                                       ...             
1290       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0   
1457       4.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  2.0  0.0   
5201       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
5952       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
5914       0.0  0.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

                                                 
MovieID 3945 3946 3947 3948 3949 3950 3951 3952  
UserID                                           
1290     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1457     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
5201     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
5952     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
5914     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 3706 columns]

In [ ]:
# Nodes for encoding layer
n_nodes_inpl = 3706
n_nodes_hl1  = 1853
n_nodes_hl2  = 925

# Nodes for hiddern layer
n_nodes_hl3  = 252

# Nodes for decoding layer
n_nodes_hl4  = 925
n_nodes_hl5  = 1853
n_nodes_outl = 3706

# input layer has 9724*4862 weights and 4862 biases
hidden_1_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_inpl,n_nodes_hl1]))}

# second encode layer has 4862*2431 weights and 2431 biases
hidden_2_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1,n_nodes_hl2]))}

# Third encode layer has 2431*512 weights and 512 biases
hidden_3_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2,n_nodes_hl3]))}

# First decode layer has 512*2431 weights and 2431 biases
hidden_4_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3,n_nodes_hl4]))}

# Second decode layer has 2431*4862 weights and 4862 biases
hidden_5_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_hl4,n_nodes_hl5]))}

# output layer has 4862*9724 weights and 9724 biases
output_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_hl5,n_nodes_outl])) }

In [ ]:
# user with 3706 ratings goes in
input_layer = tf.placeholder('float', [None, 3706])

### First Layer
# add a constant node to the first layer
# it needs to have the same shape as the input layer for me to be
# able to concatinate it later
input_layer_const = tf.fill([tf.shape(input_layer)[0], 1] ,1.0 )
# multiply output of input_layer wth a weight matrix
layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(input_layer,hidden_1_layer_vals['weights']),input_layer_const))

### Second Layer
input_layer_const1 = tf.fill( [tf.shape(layer_1)[0], 1] ,1.0  )
# multiply output of input_layer wth a weight matrix
layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1,hidden_2_layer_vals['weights']),input_layer_const1))

### Third Layer
input_layer_const2 = tf.fill( [tf.shape(layer_2)[0], 1] ,1.0  )
# multiply output of input_layer wth a weight matrix
layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2,hidden_3_layer_vals['weights']),input_layer_const2))

### Fourth Layer
input_layer_const3 = tf.fill( [tf.shape(layer_3)[0], 1] ,1.0  )
# multiply output of input_layer wth a weight matrix
layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3,hidden_4_layer_vals['weights']),input_layer_const3))

### Fifth Layer
input_layer_const4 = tf.fill( [tf.shape(layer_4)[0], 1] ,1.0  )
# multiply output of input_layer wth a weight matrix
layer_5 = tf.nn.sigmoid(tf.add(tf.matmul(layer_4,hidden_5_layer_vals['weights']),input_layer_const4))

### Output Layer
# adding one bias node to the hidden layer
layer1_const = tf.fill( [tf.shape(layer_5)[0], 1] ,1.0  )

# multiply output of hidden with a weight matrix to get final output
output_layer = tf.matmul(layer_5,output_layer_vals['weights'])

# output_true shall have the original shape for error calculations
output_true = tf.placeholder('float', [None, 3706])
# define our cost function
meansq =    tf.reduce_mean(tf.square(output_layer - output_true))
# define our optimizer
learn_rate = 0.001   # how fast the model should learn
optimizer = tf.train.AdagradOptimizer(learn_rate).minimize(meansq)

In [ ]:
# initialising variables and starting the session
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# defining batch size, number of epochs and learning rate
batch_size = 100  # how many users to use together for training
hm_epochs =100    # how many times to go through the entire dataset
tot_users = X_train_enc.shape[0] # total number of users

In [ ]:
# running the model for a 100 epochs taking 100 users in batches
# total improvement is printed out after each epoch
for epoch in range(hm_epochs):
    epoch_loss = 0    # initializing error as 0

    for i in range(int(tot_users/batch_size)):
        epoch_x = X_train_enc[ i*batch_size : (i+1)*batch_size ]
        _, c = sess.run([optimizer, meansq],feed_dict={input_layer: epoch_x,output_true: epoch_x})
        epoch_loss += c

    output_train = sess.run(output_layer,feed_dict={input_layer:X_train_enc})
    output_test = sess.run(output_layer,feed_dict={input_layer:X_test_enc})

    print('MSE train', MSE(output_train, X_train_enc),'MSE test', MSE(output_test, X_test_enc))
    print('Epoch', epoch, '/', hm_epochs, 'loss:',epoch_loss)

MSE train 765.9724014735335 MSE test 771.0713279656181
Epoch 0 / 100 loss: 41941.98718261719
MSE train 632.508892537778 MSE test 638.1463096411834
Epoch 1 / 100 loss: 33561.754150390625
MSE train 539.357895131807 MSE test 544.7478041392549
Epoch 2 / 100 loss: 28170.36248779297
MSE train 469.274934833147 MSE test 474.0867391747831
Epoch 3 / 100 loss: 24259.451080322266
MSE train 414.0845583167018 MSE test 418.7337129950744
Epoch 4 / 100 loss: 21249.199279785156
MSE train 369.6585927971956 MSE test 373.91033857767377
Epoch 5 / 100 loss: 18849.630432128906
MSE train 333.10335202343987 MSE test 337.01605215437735
Epoch 6 / 100 loss: 16902.669891357422
MSE train 302.3536936651585 MSE test 305.9054147605407
Epoch 7 / 100 loss: 15281.198455810547
MSE train 276.27345441370505 MSE test 279.54809517585556
Epoch 8 / 100 loss: 13912.96240234375
MSE train 253.87516381873337 MSE test 256.89605613279696
Epoch 9 / 100 loss: 12745.303146362305
MSE train 234.32695162307462 MSE test 237.13267374054993
Ep

In [ ]:
sample_user = X_test_enc.iloc[774,:]
sample_user_pred = sess.run(output_layer, feed_dict={input_layer:[sample_user]})

In [ ]:
temp_list = sample_user_pred[0].tolist()
temp_list.sort(reverse=True)
print(temp_list)

[11.285688400268555, 10.0316162109375, 9.339372634887695, 8.07667350769043, 7.831082820892334, 7.655648231506348, 7.5894622802734375, 7.512002944946289, 7.423049449920654, 7.3681206703186035, 7.271065711975098, 7.270009994506836, 7.138099670410156, 7.04550838470459, 6.943750858306885, 6.751956939697266, 6.708894729614258, 6.584221839904785, 6.416143894195557, 6.377815246582031, 6.340117454528809, 6.310380458831787, 6.190062046051025, 6.151238441467285, 6.01973819732666, 5.961700916290283, 5.867703437805176, 5.853249549865723, 5.754300117492676, 5.618596076965332, 5.588037967681885, 5.518455982208252, 5.492664813995361, 5.43904972076416, 5.405269622802734, 5.381048679351807, 5.3061628341674805, 5.290342330932617, 5.22867488861084, 5.2187323570251465, 5.204679012298584, 5.117154121398926, 4.8517537117004395, 4.835528373718262, 4.812573432922363, 4.77054500579834, 4.768368721008301, 4.721131324768066, 4.718137264251709, 4.5800557136535645, 4.510054588317871, 4.492406845092773, 4.458554267

In [ ]:
mock_user_id = 215


#Selecting the input user
inputUser = np.asarray(temp_list).reshape(1,-1)
inputUser[0:5]


#Feeding in the user and reconstructing the input
hidden_0 = tf.nn.sigmoid(tf.matmul(visible_0, W) + hiddenBias)
vv1 = tf.nn.sigmoid(tf.matmul(hidden_0, tf.transpose(W)) + visibleBias)
feed = sess.run(hidden_0, feed_dict={ visible_0: inputUser, W: previous_weight, hiddenBias: previous_hb})
rec = sess.run(vv1, feed_dict={ hidden_0: feed, W: previous_weight, visibleBias: previous_vb})
print(rec)


scored_movies_df_mock = movies_df[movies_df['MovieID'].isin(user_rating_df.columns)]
scored_movies_df_mock = scored_movies_df_mock.assign(RecommendationScore = rec[0])
scored_movies_df_mock.sort_values(["RecommendationScore"], ascending=False).head(20)

NameError: ignored

In [ ]:
movies_df_mock = ratings_df[ratings_df['UserID'] == mock_user_id]
movies_df_mock.head()

#Merging movies_df with ratings_df by MovieID
merged_df_mock = scored_movies_df_mock.merge(movies_df_mock, on='MovieID', how='outer')

merged_df_mock.sort_values(["RecommendationScore"], ascending=False).head(20)